In [41]:
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report 
from collections import Counter
from sklearn.utils import resample
from sklearn import preprocessing

In [42]:
test = pd.read_csv("./additional_files/test.dat", sep= ' ')
movieActors = pd.read_csv("additional_files/movie_actors.dat", sep= '\t',encoding='latin-1')
movieDirectors= pd.read_csv("additional_files/movie_directors.dat", sep= '\t',encoding='latin-1')
movieGenres= pd.read_csv("additional_files/movie_genres.dat", sep= '\t',encoding='latin-1')
movieTags= pd.read_csv("additional_files/movie_tags.dat", sep= '\t',encoding='latin-1')
tags= pd.read_csv("additional_files/tags.dat", sep= '\t',encoding='latin-1')
trnTst= pd.read_csv("additional_files/test.dat", sep= ' ',encoding='latin-1')
train= pd.read_csv("additional_files/train.dat", sep= ' ',encoding='latin-1')
userTaggedMovies= pd.read_csv("additional_files/user_taggedmovies.dat", sep= ' ',encoding='latin-1')

In [43]:
# minmax scaling of userid and movieid (because very huge numbers)
df = train[['userID','movieID']]
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.columns=['userID','movieID']
train[['userID','movieID']] = df[['userID','movieID']]

In [44]:
train["movieID"].value_counts()

0.039458    1259
0.076644    1221
0.005450    1171
0.091368    1139
0.004529    1132
            ... 
0.138273       1
0.121753       1
0.484800       1
0.113738       1
0.027913       1
Name: movieID, Length: 9936, dtype: int64

In [45]:
train

,userID,movieID,rating
0,0.0,0.000031,1.0
1,0.0,0.000476,4.5
2,0.0,0.001674,4.0
3,0.0,0.002487,4.0
4,0.0,0.002518,4.5
...,...,...,...
641694,1.0,0.658647,4.0
641695,1.0,0.684057,4.0
641696,1.0,0.715117,4.0
641697,1.0,0.937696,5.0


In [46]:
test

,userID,movieID
0,75,653
1,75,1527
2,75,3258
3,75,7007
4,78,82
...,...,...
71294,71534,2208
71295,71534,2997
71296,71534,4306
71297,71534,7132


In [47]:
df = test[['userID','movieID']]
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df.columns=['userID','movieID']
test[['userID','movieID']] = df[['userID','movieID']]

In [48]:
test

,userID,movieID
0,0.000000,0.010017
1,0.000000,0.023444
2,0.000000,0.050038
3,0.000000,0.107636
4,0.000042,0.001244
...,...,...
71294,1.000000,0.033907
71295,1.000000,0.046029
71296,1.000000,0.066139
71297,1.000000,0.109556


In [49]:
train

,userID,movieID,rating
0,0.0,0.000031,1.0
1,0.0,0.000476,4.5
2,0.0,0.001674,4.0
3,0.0,0.002487,4.0
4,0.0,0.002518,4.5
...,...,...,...
641694,1.0,0.658647,4.0
641695,1.0,0.684057,4.0
641696,1.0,0.715117,4.0
641697,1.0,0.937696,5.0


In [50]:
# chose random n samples to train and assign to df (since dataset is pretty huge)
df = train.sample(n=100000)

In [51]:
# y_train = rating column in df
y_train = df.values[:,2]
y_train

array([5. , 5. , 4.5, ..., 0.5, 4.5, 4. ])

In [52]:
# remove rating from df
df = df.drop(columns=['rating'])
X = df
X

,userID,movieID
430648,0.651646,0.000752
653,0.001399,0.095452
473283,0.717866,0.041700
96307,0.151555,0.007661
525727,0.811822,0.054873
...,...,...
496824,0.748541,0.506709
350435,0.517248,0.730962
208350,0.326243,0.039105
76619,0.120013,0.019668


In [53]:
# df = finaltest[['userID','movieID']]
# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(x_scaled)
# df.columns=['userID','movieID']
# finaltest[['userID','movieID']] = df[['userID','movieID']]

In [54]:
# train test split
X_train = X.values[:,:]
# y_train = df.values[:,2]
X_test =  test.values[:,:]
y_train

array([5. , 5. , 4.5, ..., 0.5, 4.5, 4. ])

In [55]:
X_train

array([[6.51646399e-01, 7.52318369e-04],
       [1.39940385e-03, 9.54523122e-02],
       [7.17866189e-01, 4.16999324e-02],
       ...,
       [3.26243020e-01, 3.91052017e-02],
       [1.20012875e-01, 1.96677516e-02],
       [3.37900055e-01, 8.99066511e-01]])

In [56]:
X_test

array([[0.        , 0.0100169 ],
       [0.        , 0.02344446],
       [0.        , 0.05003841],
       ...,
       [1.        , 0.06613919],
       [1.        , 0.109556  ],
       [1.        , 0.5505454 ]])

In [57]:
type(trn)

pandas.core.frame.DataFrame

In [18]:
# fintrn.to_csv('train_dataset_processed.csv', index=False, header=True)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cos_sim = cosine_similarity(X_train, X_test)

In [26]:
cos_sim.shape

(50000, 71299)

In [21]:
X_train.shape[0]

100000

In [27]:
X_test.shape

(71299, 2)

In [21]:
n = 1000

In [25]:
X_train[20000:40000].shape

(20000, 2)

In [ ]:
cos_sim2 = cosine_similarity(X_train[10000:20000], X_test)
cos_sim2

In [ ]:
cos_sim_new = np.concatenate((cos_sim, cos_sim2))

In [ ]:
cos_sim_new.shape

In [ ]:
cos_sim_new

In [22]:
cos_sim = np.empty([0, 71299])

In [23]:
cos_sim.shape

(0, 71299)

In [49]:
cos_sim

array([[0.20518221, 0.20518221, 0.20518221, ..., 0.99013113, 0.99524781,
        0.95633224],
       [0.03096669, 0.03096669, 0.03096669, ..., 0.99938506, 0.99694791,
        0.89052875],
       [0.00890867, 0.00890867, 0.00890867, ..., 0.99836828, 0.99498298,
        0.88027584],
       ...,
       [0.02584856, 0.02584856, 0.02584856, ..., 0.99919243, 0.99653511,
        0.88818777],
       [0.04652293, 0.04652293, 0.04652293, ..., 0.99980982, 0.99804244,
        0.89750289],
       [0.13512875, 0.13512875, 0.13512875, ..., 0.99758582, 0.99965094,
        0.93314996]])

In [25]:
for i in range(0,10000, n):
    cos_sim_new = cosine_similarity(X_train[i:i+n], X_test)
    cos_sim = np.concatenate((cos_sim, cos_sim_new))
    print(cos_sim.shape)

(1000, 71299)
(2000, 71299)
(3000, 71299)
(4000, 71299)
(5000, 71299)
(6000, 71299)
(7000, 71299)
(8000, 71299)
(9000, 71299)
(10000, 71299)


In [28]:
cos_sim = cos_sim.transpose()

In [29]:
indices = list()
k = 20 # Best value of k chosen after experimenting
for doc in cos_sim:
    indices.append(np.argsort(doc)[-k:])

In [30]:
print(len(indices))

71299


In [33]:
pred_val = list()
for arr in indices:
    pred_val.append(np.mean([y_train[i] for i in arr]))

In [34]:
pred_val

[3.9,
 3.9,
 3.9,
 3.9,
 3.15,
 3.625,
 3.575,
 3.775,
 3.675,
 3.7,
 3.575,
 3.575,
 3.525,
 3.525,
 3.625,
 3.625,
 3.8,
 3.825,
 3.8,
 3.775,
 3.825,
 3.85,
 3.875,
 3.875,
 3.875,
 3.825,
 3.85,
 3.775,
 3.775,
 3.775,
 3.775,
 3.875,
 3.875,
 3.875,
 3.925,
 3.9,
 3.9,
 3.9,
 3.9,
 3.9,
 3.9,
 3.9,
 3.9,
 3.775,
 3.775,
 3.875,
 3.65,
 3.45,
 2.925,
 3.275,
 3.15,
 3.7,
 3.65,
 3.55,
 3.675,
 3.575,
 3.6,
 3.425,
 3.475,
 3.575,
 3.925,
 4.025,
 3.875,
 3.375,
 3.7,
 3.575,
 3.225,
 3.625,
 3.9,
 3.4,
 3.725,
 3.8,
 3.925,
 3.925,
 3.625,
 3.75,
 3.425,
 3.35,
 3.65,
 3.3,
 3.3,
 3.55,
 3.2,
 3.675,
 3.375,
 3.525,
 3.525,
 3.125,
 3.925,
 3.7,
 3.425,
 3.65,
 3.6,
 3.775,
 3.825,
 3.875,
 3.625,
 3.775,
 3.675,
 3.625,
 3.825,
 3.35,
 3.7,
 3.6,
 3.925,
 3.425,
 3.35,
 3.175,
 3.525,
 3.375,
 3.275,
 3.65,
 3.425,
 3.375,
 3.6,
 3.425,
 3.875,
 3.325,
 3.175,
 3.45,
 3.6,
 3.725,
 3.25,
 3.675,
 3.55,
 3.875,
 3.325,
 3.2,
 3.725,
 3.95,
 3.7,
 3.4,
 3.625,
 3.7,
 3.45,
 3.775,
 

In [35]:
len(pred_val)

71299

In [36]:
pred_formatted_list = [ round(elem,1) for elem in pred_val ]

In [37]:
pred_df = pd.DataFrame(pred_formatted_list,columns=['Column_Name'])
pred_df

,Column_Name
0,3.9
1,3.9
2,3.9
3,3.9
4,3.2
...,...
71294,3.4
71295,3.6
71296,3.2
71297,3.5


In [39]:
pred_df['Column_Name'].value_counts(sort=False)

3.5     8866
2.3        6
2.4        7
2.5        7
2.8      537
2.7      117
3.8     4933
3.0     2600
3.1     3006
3.6    12519
4.0      757
4.2       24
3.9     1285
4.1      135
3.2     8479
3.3     7472
3.4    14582
2.6       66
3.7     5201
2.9      700
Name: Column_Name, dtype: int64

In [40]:
with open('pred_values_knn_cossim.txt', 'w') as filehandle:
    for listitem in pred_formatted_list:
        filehandle.write('%s\n' % listitem)

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)

In [58]:
from sklearn.neighbors import KNeighborsRegressor
regressor = KNeighborsRegressor(n_neighbors=140)
regressor.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=140)

In [59]:
y_pred = regressor.predict(X_test)

In [60]:
len(y_pred)

71299

In [ ]:
y_pred

In [ ]:
test["rating"] = list(y_pred)

In [ ]:
test

In [ ]:
test

In [ ]:
pred_list = list(test["rating"])

In [ ]:
pred_list

In [ ]:
test["rating"].value_counts()

In [ ]:
pred_list[10]

In [ ]:
pred_formatted_list = [ round(elem,1) for elem in pred_list ]

In [ ]:
pred_formatted_list

In [ ]:
len(pred_formatted_list)

In [ ]:
pred_df = pd.DataFrame(pred_formatted_list,columns=['Column_Name'])
pred_df

In [ ]:
pred_df['Column_Name'].value_counts(sort=False)

In [ ]:
with open('pred_values_basic.txt', 'w') as filehandle:
    for listitem in pred_formatted_list:
        filehandle.write('%s\n' % listitem)